# Convolutional Neural Network

### Importing the libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.10.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

- 데이터 증강

 비슷한 특징을 가진 데이터를 계속 학습하게 되면 성능은 올라가지만 데이터의 특징에 익숙해지는 "과적합"이 일어나게 된다.
따라서 이를 방지해주기 위해 이미지를 대칭, 전단 등 여러가지 방식으로 바꾸어주는 과정이 필요하다

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,     # 0 ~ 255사이의 픽셀 값들을 0에서 1로 바꿔주는 과정
                                   shear_range = 0.2,    
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # 이미지의 사이즈를 64by64로 줄여주는 과정
                                                 batch_size = 32,          
                                                 class_mode = 'binary')  # 출력값이 두개이기 때문에 이진법 사용

### Preprocessing the Test set

In [ ]:
# 데이터 증강을 제외한 과정은 훈련 세트와 같다
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) #kernel_size: filter의 사이즈를 결정, input_shape:(사이즈, 차원)

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) # Max Pooling 기법을 사용

### Adding a second convolutional layer

In [ ]:
# 사이즈를 결정하는 parameter외에는 동일
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) # 이미지의 특징을 정확하기 파악하기 위해서는 ANN보다 많은 unit이 필요하다

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # 출력값이 0 아니면 1이기 때문에 unit의 수는 1이다

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)